## Generation of the GT for Training

In [1]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import open3d as o3d
from scipy import spatial

from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map

from sphere import Sphere
from visualize import Visualize
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
export_ds = '/media/scratch/berlukas/nuscenes'
sem_clouds_filename = f"{export_ds}/clouds.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/clouds.npy
Shape of sem clouds is (4766, 3, 400, 400)


In [3]:
n_clouds = sem_cloud_features.shape[0]
idx_sem = 2
classes = [None] * n_clouds
for i in tqdm(range(0, n_clouds)):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (3, -1)).T    
    classes[i] = sem_cloud[:,idx_sem]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
n_unique_classes = len(np.unique(classes))
print(f"we have {n_unique_classes} classes in the dataset.")


shape of classes is (4766, 160000)
we have 32 classes in the dataset.


## Iterate over the features and generate the GT

In [4]:
def map_nuscenes_label(label):
    noise = []
    vehicle = [1,2,3,4,5,6,7,8,9,10,11]
    living = [12,13,14,15,16,17,18,19]
    objects = [20,21,22,23]
    ground = [24,25,26,27]
    buildings = [28]
    vegetation = [29]
    other = [30,0]
    ego = [31]
    
    if label in noise:
        return 0
    elif label in vehicle:
        return 1
    elif label in living:
        return 2
    elif label in objects:
        return 3
    elif label in ground:
        return 4
    elif label in buildings:
        return 5
    elif label in vegetation:
        return 6
    elif label in other:
        return 7
    elif label in ego:
        return 8
    else:
        return 0

In [5]:
n_data = sem_cloud_features.shape[0]
n_features = sem_cloud_features.shape[1]
n_rows = sem_cloud_features.shape[2]
n_cols = sem_cloud_features.shape[3]

reshaped_gt = np.zeros((n_data, n_features, n_rows, n_cols))
for feature in tqdm(range(n_data)):
    for i in range(n_rows):
        for j in range(n_cols):
            cur_class = int(sem_cloud_features[feature, idx_sem, i, j])
            mapped_class = map_nuscenes_label(cur_class)
            reshaped_gt[feature, idx_sem, i, j] = mapped_class

In [6]:
sem_clouds_gt_filename = f"{export_ds}/sampled_clouds_fixed_sem.npy"
np.save(sem_clouds_gt_filename, reshaped_gt)
print(f"Wrote features to {sem_clouds_gt_filename}")
print(f"With final shape: {reshaped_gt.shape}")

Wrote features to /media/scratch/berlukas/nuscenes/sampled_clouds_fixed_sem.npy
With final shape: (4766, 3, 400, 400)


In [4]:
np.count_nonzero(classes[:,:] == 19) / np.count_nonzero(classes[:,:] > 0)

0.0

In [5]:
np.count_nonzero(classes[:,:] == 19)

0

In [17]:
np.count_nonzero(classes[:,:] == 0)

178539

In [9]:
np.count_nonzero(sem_cloud_features[:, 1, :, :] == 0)

178539